## 🤖 Text Classification with BERT (AG News + Transformers)
This notebook shows how to fine-tune a pre-trained BERT model on the AG News dataset using Hugging Face Transformers.

In [ ]:
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score
from tqdm import tqdm

### 1. Load the AG News Dataset

In [ ]:
# Load dataset from Hugging Face Datasets
dataset = load_dataset("ag_news")
print(dataset)

### 2. Tokenize with BERT Tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True)

# Tokenize datasets
tokenized = dataset.map(tokenize, batched=True)
tokenized = tokenized.rename_column("label", "labels")
tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Data collator for dynamic padding
collator = DataCollatorWithPadding(tokenizer=tokenizer)

### 3. Prepare Model and DataLoaders

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_loader = DataLoader(tokenized["train"].select(range(2000)), batch_size=16, shuffle=True, collate_fn=collator)
test_loader = DataLoader(tokenized["test"].select(range(500)), batch_size=32, shuffle=False, collate_fn=collator)

### 4. Train the BERT Classifier

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)

model.train()
for epoch in range(3):
    total_loss = 0
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

### 5. Evaluate the BERT Model

In [ ]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        labels = batch["labels"]
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=1).cpu()
        all_preds.extend(preds)
        all_labels.extend(labels)

acc = accuracy_score(all_labels, all_preds)
print("Test Accuracy:", acc)